<a href="https://colab.research.google.com/github/ahmedabdulghany7/Competitive-Intelligence-Multi-Agent-System/blob/main/Strategic_Analyst_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Strategic Analyst Agent (LLM-Powered) for Competitive Intelligence System

This agent analyzes "Why does this matter?" by providing strategic context,
threat/opportunity assessment, and connecting events to broader trends.

Core Functions:
1. Strategic Context Analysis
2. Threat and Opportunity Assessment
3. Trend Connection and Pattern Recognition
4. Competitive Positioning Analysis
5. Market Implications Assessment
"""

'\nStrategic Analyst Agent (LLM-Powered) for Competitive Intelligence System\n\nThis agent analyzes "Why does this matter?" by providing strategic context,\nthreat/opportunity assessment, and connecting events to broader trends.\n\nCore Functions:\n1. Strategic Context Analysis\n2. Threat and Opportunity Assessment\n3. Trend Connection and Pattern Recognition\n4. Competitive Positioning Analysis\n5. Market Implications Assessment\n'

In [2]:
pip install -U "langchain>=0.2" "langchain-openai>=0.2" "langchain-community>=0.2" pydantic>=2 openai>=1.30

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from enum import Enum
from datetime import datetime, timezone
import logging
import json
import asyncio
import re

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

PRICE_PER_1K = {
    "gpt-4o-mini":   {"input": 0.150, "output": 0.600},
    "gpt-4o":        {"input": 2.500, "output": 5.000},
    "gpt-4.1-mini":  {"input": 0.300, "output": 1.200},
    "gpt-4":         {"input": 3.000, "output": 6.000},
}
def _estimate_cost(model: str, usage: dict) -> float:
    pr = PRICE_PER_1K.get(model, None)
    if not pr:
        return 0.0
    in_t  = usage.get("input_tokens", 0) or usage.get("prompt_tokens", 0) or 0
    out_t = usage.get("output_tokens", 0) or usage.get("completion_tokens", 0) or 0
    return (in_t/1000.0)*pr["input"] + (out_t/1000.0)*pr["output"]

In [5]:
from google.colab import userdata
import openai

openai.api_key = userdata.get('OPENAI_API_KEY')

In [6]:
class AnalysisType(Enum):
    STRATEGIC_CONTEXT = "strategic_context"
    THREAT_ASSESSMENT = "threat_assessment"
    OPPORTUNITY_ASSESSMENT = "opportunity_assessment"
    TREND_ANALYSIS = "trend_analysis"
    COMPETITIVE_POSITIONING = "competitive_positioning"
    MARKET_IMPLICATIONS = "market_implications"

class ThreatLevel(Enum):
    CRITICAL = "critical"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"
    NEGLIGIBLE = "negligible"

class OpportunityLevel(Enum):
    BREAKTHROUGH = "breakthrough"
    HIGH = "high"
    MEDIUM = "medium"
    LOW = "low"
    NEGLIGIBLE = "negligible"

@dataclass
class StrategicInsight:
    analysis_type: AnalysisType
    insight: str
    confidence_score: float  # 0-1
    supporting_evidence: List[str]
    implications: List[str]
    timeline: str
    certainty_level: str

@dataclass
class ThreatAssessment:
    threat_level: ThreatLevel
    threat_description: str
    potential_impact: str
    affected_areas: List[str]
    mitigation_urgency: str
    confidence_score: float

@dataclass
class OpportunityAssessment:
    opportunity_level: OpportunityLevel
    opportunity_description: str
    potential_value: str
    required_capabilities: List[str]
    time_sensitivity: str
    confidence_score: float

@dataclass
class CompetitiveContext:
    our_strengths: List[str]
    our_weaknesses: List[str]
    market_position: str
    key_differentiators: List[str]
    strategic_priorities: List[str]
    recent_initiatives: List[str]

    focus_regions: List[str] = None           # ["EG", "KSA", "UAE", "IN", "EU", "US"]
    target_price_bands: List[str] = None      # ["budget","mid-range","flagship"]
    sales_channels: List[str] = None          # ["carrier","retail","ecom"]
    operator_partners: List[str] = None       # ["Vodafone","Orange","Etisalat","STC",...]

@dataclass
class StrategicAnalysis:
    signal_id: str
    competitor: str
    event_summary: str
    strategic_context: str
    threat_assessment: Optional[ThreatAssessment]
    opportunity_assessment: Optional[OpportunityAssessment]
    key_insights: List[StrategicInsight]
    broader_trends: List[str]
    competitive_implications: str
    strategic_recommendations: List[str]
    analysis_timestamp: datetime
    llm_cost: float
    tokens_used: int


In [7]:
class StrategicAnalystAgent:
    """
    Strategic Analyst Agent (LLM-powered) for Smartphone OEMs (Samsung, Apple, Xiaomi, Oppo, Realme, etc.)
    Answers:
      - Why does this event matter for us?
      - What should we do now (actionable recommendations)?
    """

    def __init__(
        self,
        llm_model: str = "gpt-4o-mini",
        temperature: float = 0.2,
        max_tokens: int = 1600,
        competitive_context: Optional[CompetitiveContext] = None,
        historical_analysis: Optional[List[StrategicAnalysis]] = None,
        impact_threshold: float = 6.0,
    ):
        self.llm = ChatOpenAI(model=llm_model, temperature=temperature, max_tokens=max_tokens)
        self._llm_model_name = llm_model
        self.competitive_context = competitive_context or self._default_mobile_context()
        self.historical_analysis = historical_analysis or []
        self.impact_threshold = impact_threshold
        self._init_prompt_templates()
        self._last_usage = {}

        self.analysis_config = {
            'include_threat_assessment': True,
            'include_opportunity_assessment': True,
            'include_trend_analysis': True,
            'max_historical_context': 10
        }


    def _default_mobile_context(self) -> CompetitiveContext:
        return CompetitiveContext(
            our_strengths=["camera_quality","battery_life","fast_updates","after_sales_service"],
            our_weaknesses=["smaller_marketing_budget","limited_operator_deals"],
            market_position="value_midrange_challenger",
            key_differentiators=["clean_android","price_to_spec","regional_support"],
            strategic_priorities=["operator_partnerships","retail_expansion","flagship_camera_push"],
            recent_initiatives=["ecom_bundle_program","trade_in_campaign","service_centers_expansion"],
            focus_regions=["EG","KSA","UAE","IN","EU"],
            target_price_bands=["budget","mid-range","flagship"],
            sales_channels=["carrier","retail","ecom"],
            operator_partners=["Vodafone","Orange","Etisalat","STC","du"]
        )


    def _init_prompt_templates(self):
        self.strategic_analysis_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a senior strategic analyst for a smartphone OEM in global markets (MENA, India, EU, US).
    Return ONLY valid JSON (no prose) with keys:
    {{
      "strategic_context": "string",
      "competitive_implications": "string",
      "key_insights": ["string", "..."],
      "recommendations": ["string", "..."]
    }}
    Guidance:
    - Ground strictly in the event text and the provided company context.
    - Consider smartphone-specific drivers: price bands, chipset/performance, camera, battery, display, operator/carrier deals, retail/ecom channels, launch waves, regional promos, trade-in, warranty/after-sales, brand halo, ecosystem lock-in (iOS/Android), 5G bands, certification.
    - Be concise and actionable (max ~3-4 sentences per string)."""),
            ("human", """COMPANY CONTEXT:
    - Strengths: {our_strengths}
    - Weaknesses: {our_weaknesses}
    - Market Position: {market_position}
    - Differentiators: {key_differentiators}
    - Strategic Priorities: {strategic_priorities}
    - Recent Initiatives: {recent_initiatives}
    - Focus Regions: {focus_regions}
    - Target Price Bands: {target_price_bands}
    - Sales Channels: {sales_channels}
    - Operator Partners: {operator_partners}

    EVENT TO ANALYZE:
    - Competitor: {competitor}
    - Event Type: {event_type}
    - Description: {event_description}
    - Impact Score: {impact_score}/10
    - Source: {source}
    - Date: {event_date}

    HISTORICAL CONTEXT:
    {historical_context}""")
        ])

        self.threat_assessment_prompt = ChatPromptTemplate.from_messages([
            ("system", """You assess competitive threats in the smartphone market.
    Return ONLY valid JSON:
    {{
      "threat_level": "CRITICAL|HIGH|MEDIUM|LOW|NEGLIGIBLE",
      "threat_description": "string",
      "potential_impact": "string",
      "affected_areas": ["market_share","pricing","channel_conflicts","brand_perception","retail_footprint","operator_relations"],
      "mitigation_urgency": "Immediate|High|Medium|Low",
      "confidence": 0-100
    }}"""),
            ("human", """Evaluate threat:
    Event: {event_description}
    Competitor: {competitor}
    Our Weaknesses: {our_weaknesses}
    Our Strengths: {our_strengths}""")
        ])

        self.opportunity_assessment_prompt = ChatPromptTemplate.from_messages([
            ("system", """You identify opportunities in the smartphone market.
    Return ONLY valid JSON:
    {{
      "opportunity_level": "BREAKTHROUGH|HIGH|MEDIUM|LOW|NEGLIGIBLE",
      "opportunity_description": "string",
      "potential_value": "string",
      "required_capabilities": ["product","pricing","channel","ops","partnerships"],
      "time_sensitivity": "Immediate|This quarter|Next 2 quarters|Longer",
      "confidence": 0-100
    }}"""),
            ("human", """Evaluate opportunity:
    Event: {event_description}
    Competitor: {competitor}
    Our Capabilities: {our_strengths}
    Strategic Priorities: {strategic_priorities}""")
        ])

        self.trend_prompt_template = """List 3-5 bullet points (no prose) of broader smartphone industry trends connected to:
    Event: {event_text}
    Context: {historical_context}
    Focus on: chipset cycles, camera race, pricing wars, operator deals, retail dominance, iOS/Android ecosystem moats, 5G/regulatory shifts, refurb/trade-in programs.
    Return as '- <point>' lines only."""


    async def analyze_signal(self, signal: Dict[str, Any]) -> StrategicAnalysis:
        try:
            logger.info(f"Starting strategic analysis for signal: {signal.get('id', 'unknown')}")
            total_tokens = 0
            total_cost = 0.0

            historical_context = self._prepare_historical_context(signal.get('competitor'))

            main_analysis = await self._perform_main_analysis(signal, historical_context)
            usage_main = getattr(self, "_last_usage", {}) or {}
            total_tokens += usage_main.get("total_tokens", 0)
            total_cost   += _estimate_cost(self._llm_model_name, usage_main)


            run_deep = (signal.get("event_type") in {"launch","pricing","product_launch","price_change"}) or \
                       (float(signal.get("impact_score", 0)) >= self.impact_threshold)

            threat_assessment = None
            if self.analysis_config['include_threat_assessment'] and run_deep:
                threat_assessment = await self._assess_threats(signal)
                usage_threat = getattr(self, "_last_usage", {}) or {}
                total_tokens += usage_threat.get("total_tokens", 0)
                total_cost   += _estimate_cost(self._llm_model_name, usage_threat)

            opportunity_assessment = None
            if self.analysis_config['include_opportunity_assessment'] and run_deep:
                opportunity_assessment = await self._assess_opportunities(signal)
                usage_opp = getattr(self, "_last_usage", {}) or {}
                total_tokens += usage_opp.get("total_tokens", 0)
                total_cost   += _estimate_cost(self._llm_model_name, usage_opp)

            broader_trends = []
            if self.analysis_config['include_trend_analysis'] and run_deep:
                broader_trends = await self._analyze_trends(signal, historical_context)
                usage_trend = getattr(self, "_last_usage", {}) or {}
                total_tokens += usage_trend.get("total_tokens", 0)
                total_cost   += _estimate_cost(self._llm_model_name, usage_trend)

            strategic_analysis = StrategicAnalysis(
                signal_id=signal.get('id', 'unknown'),
                competitor=signal.get('competitor', 'unknown'),
                event_summary=self._create_event_summary(signal),
                strategic_context=main_analysis.get('strategic_context', ''),
                threat_assessment=threat_assessment,
                opportunity_assessment=opportunity_assessment,
                key_insights=self._to_insights(main_analysis.get('key_insights', [])),
                broader_trends=broader_trends,
                competitive_implications=main_analysis.get('competitive_implications', ''),
                strategic_recommendations=main_analysis.get('recommendations', []),
                analysis_timestamp=datetime.now(timezone.utc),
                llm_cost=total_cost,
                tokens_used=total_tokens
            )

            self.historical_analysis.append(strategic_analysis)
            if len(self.historical_analysis) > self.analysis_config['max_historical_context']:
                self.historical_analysis = self.historical_analysis[-self.analysis_config['max_historical_context']:]

            logger.info(f"Strategic analysis completed. Cost~ ${total_cost:.4f}, Tokens: {total_tokens}")
            return strategic_analysis

        except Exception as e:
            logger.error(f"Error in strategic analysis: {str(e)}")
            return self._create_error_analysis(signal, str(e))


    async def _perform_main_analysis(self, signal: Dict[str, Any], historical_context: str) -> Dict[str, Any]:
        pv = self._prompt_vars(signal, historical_context)
        msgs = self.strategic_analysis_prompt.format_messages(**pv)
        resp: AIMessage = await self.llm.ainvoke(msgs)
        self._last_usage = resp.usage_metadata or {}
        raw = resp.content or ""

        data = self._safe_json_parse(raw)

        return {
            "strategic_context": data.get("strategic_context", "") or "",
            "competitive_implications": data.get("competitive_implications", "") or "",
            "key_insights": data.get("key_insights", []) or [],
            "recommendations": data.get("recommendations", []) or []
        }


    async def _assess_threats(self, signal: Dict[str, Any]) -> ThreatAssessment:
        pv = {
            'event_description': signal.get('text', 'No description'),
            'competitor': signal.get('competitor', 'Unknown'),
            'our_weaknesses': ', '.join(self.competitive_context.our_weaknesses),
            'our_strengths':  ', '.join(self.competitive_context.our_strengths)
        }
        msgs = self.threat_assessment_prompt.format_messages(**pv)
        resp: AIMessage = await self.llm.ainvoke(msgs)
        self._last_usage = resp.usage_metadata or {}
        data = self._safe_json_parse(resp.content)


        level = (data.get("threat_level","MEDIUM") or "MEDIUM").upper()
        tl = ThreatLevel[level] if level in ThreatLevel.__members__ else ThreatLevel.MEDIUM
        conf = float(data.get("confidence", 70) or 70)/100.0

        affected = data.get("affected_areas") or ["market_share","pricing"]
        if isinstance(affected, str): affected = [affected]

        return ThreatAssessment(
            threat_level=tl,
            threat_description=str(data.get("threat_description",""))[:500],
            potential_impact=str(data.get("potential_impact","")),
            affected_areas=affected,
            mitigation_urgency=str(data.get("mitigation_urgency","Medium")),
            confidence_score=conf
        )

    async def _assess_opportunities(self, signal: Dict[str, Any]) -> OpportunityAssessment:
        pv = {
            'event_description': signal.get('text', 'No description'),
            'competitor': signal.get('competitor', 'Unknown'),
            'our_strengths': ', '.join(self.competitive_context.our_strengths),
            'strategic_priorities': ', '.join(self.competitive_context.strategic_priorities)
        }
        msgs = self.opportunity_assessment_prompt.format_messages(**pv)
        resp: AIMessage = await self.llm.ainvoke(msgs)
        self._last_usage = resp.usage_metadata or {}
        data = self._safe_json_parse(resp.content)

        level = (data.get("opportunity_level","MEDIUM") or "MEDIUM").upper()
        ol = OpportunityLevel[level] if level in OpportunityLevel.__members__ else OpportunityLevel.MEDIUM
        conf = float(data.get("confidence", 70) or 70)/100.0

        req_caps = data.get("required_capabilities") or ["product","pricing","channel"]
        if isinstance(req_caps, str): req_caps = [req_caps]

        return OpportunityAssessment(
            opportunity_level=ol,
            opportunity_description=str(data.get("opportunity_description",""))[:500],
            potential_value=str(data.get("potential_value","")),
            required_capabilities=req_caps,
            time_sensitivity=str(data.get("time_sensitivity","This quarter")),
            confidence_score=conf
        )

    async def _analyze_trends(self, signal: Dict[str, Any], historical_context: str) -> List[str]:
        tp = self.trend_prompt_template.format(
            event_text=signal.get('text','No description'),
            historical_context=historical_context
        )
        resp: AIMessage = await self.llm.ainvoke([HumanMessage(content=tp)])
        self._last_usage = resp.usage_metadata or {}

        lines = [ln.strip() for ln in resp.content.splitlines() if ln.strip().startswith("-")]
        return [ln.lstrip("-").strip() for ln in lines][:5]




    def _prompt_vars(self, signal: Dict[str, Any], historical_context: str) -> Dict[str, Any]:
        return {
            'competitor': signal.get('competitor', 'Unknown'),
            'event_type': signal.get('event_type', 'Unknown'),
            'event_description': signal.get('text', 'No description available'),
            'impact_score': signal.get('impact_score', 0),
            'source': signal.get('source', 'Unknown'),
            'event_date': (signal.get('timestamp') or datetime.now(timezone.utc)).strftime('%Y-%m-%d'),
            'historical_context': historical_context,
            'our_strengths': ', '.join(self.competitive_context.our_strengths),
            'our_weaknesses': ', '.join(self.competitive_context.our_weaknesses),
            'market_position': self.competitive_context.market_position,
            'key_differentiators': ', '.join(self.competitive_context.key_differentiators),
            'strategic_priorities': ', '.join(self.competitive_context.strategic_priorities),
            'recent_initiatives': ', '.join(self.competitive_context.recent_initiatives),
            'focus_regions': ', '.join(self.competitive_context.focus_regions or []),
            'target_price_bands': ', '.join(self.competitive_context.target_price_bands or []),
            'sales_channels': ', '.join(self.competitive_context.sales_channels or []),
            'operator_partners': ', '.join(self.competitive_context.operator_partners or []),
        }

    def _safe_json_parse(self, text: str) -> Dict[str, Any]:
        """
        Robust JSON extractor:
        - Strips code fences ``` and ```json
        - Finds the widest {...} block
        - Tries json.loads multiple times
        - Never raises: returns minimal dict on failure
        """
        if not text:
            return {"strategic_context":"","competitive_implications":"","key_insights":[],"recommendations":[]}

        s = text.strip()

        if s.startswith("```"):
            s = s.strip("`")
            s = s.replace("json\n", "").replace("JSON\n", "")
            s = s.strip()

        if s.lower().startswith("json"):
            s = s[4:].strip()

        try:
            return json.loads(s)
        except Exception:
            pass

        try:
            start = s.index("{")
            end   = s.rindex("}") + 1
            candidate = s[start:end]
            try:
                return json.loads(candidate)
            except Exception:
                candidate2 = candidate.replace("```", "").strip()
                candidate2 = re.sub(r",\s*([}\]])", r"\1", candidate2)
                return json.loads(candidate2)
        except Exception:
            return {"strategic_context":"","competitive_implications":"","key_insights":[],"recommendations":[]}

    def _to_insights(self, items: List[str]) -> List[StrategicInsight]:
        out = []
        for s in items:
            out.append(StrategicInsight(
                analysis_type=AnalysisType.STRATEGIC_CONTEXT,
                insight=s,
                confidence_score=0.7,
                supporting_evidence=[],
                implications=[],
                timeline="near_term",
                certainty_level="medium"
            ))
        return out

    def _prepare_historical_context(self, competitor: str) -> str:
        relevant = [a for a in self.historical_analysis[-5:] if a.competitor == competitor]
        if not relevant:
            return "No significant historical context available for this competitor."
        parts = []
        for a in relevant:
            parts.append(
                f"{a.analysis_timestamp.strftime('%Y-%m-%d')}: {a.event_summary[:140]} ..."
            )
        return "Recent competitive history:\n" + "\n".join(parts)

    def _create_event_summary(self, signal: Dict[str, Any]) -> str:
        return (
            f"{signal.get('competitor','Unknown competitor')} "
            f"{str(signal.get('event_type','event')).replace('_',' ')}: "
            f"{signal.get('text','No description')[:200]}..."
        )

    def _create_error_analysis(self, signal: Dict[str, Any], error: str) -> StrategicAnalysis:
        return StrategicAnalysis(
            signal_id=signal.get('id', 'error'),
            competitor=signal.get('competitor', 'Unknown'),
            event_summary=f"Analysis failed: {error}",
            strategic_context="Unable to analyze due to error",
            threat_assessment=None,
            opportunity_assessment=None,
            key_insights=[],
            broader_trends=[],
            competitive_implications="Analysis unavailable due to error",
            strategic_recommendations=["Review and retry analysis"],
            analysis_timestamp=datetime.now(timezone.utc),
            llm_cost=0.0,
            tokens_used=0
        )

    def update_competitive_context(self, context: CompetitiveContext):
        self.competitive_context = context
        logger.info("Updated competitive context for strategic analysis")

    def get_analysis_summary(self) -> Dict[str, Any]:
        if not self.historical_analysis:
            return {"message": "No analyses performed yet"}
        recent = self.historical_analysis[-10:]
        return {
            "total_analyses": len(self.historical_analysis),
            "recent_analyses": len(recent),
            "total_cost": sum(a.llm_cost for a in recent),
            "total_tokens": sum(a.tokens_used for a in recent),
            "competitors_analyzed": list(set(a.competitor for a in recent)),
            "avg_analysis_cost": sum(a.llm_cost for a in recent) / len(recent) if recent else 0
        }



In [ ]:
our_context = CompetitiveContext(
    our_strengths=["camera_quality","battery_life","fast_updates","after_sales_service"],
    our_weaknesses=["limited_operator_deals","smaller_marketing_budget"],
    market_position="value_midrange_challenger",
    key_differentiators=["clean_android","price_to_spec","regional_support"],
    strategic_priorities=["operator_partnerships","retail_expansion","flagship_camera_push"],
    recent_initiatives=["ecom_bundle_program","trade_in_campaign","service_centers_expansion"],
    focus_regions=["EG","KSA","UAE","IN","EU"],
    target_price_bands=["budget","mid-range","flagship"],
    sales_channels=["carrier","retail","ecom"],
    operator_partners=["Vodafone","Orange","Etisalat","STC","du"]
)

analyst = StrategicAnalystAgent(
    llm_model="gpt-4o-mini",
    competitive_context=our_context,
    impact_threshold=6.0
)


test_signal = {
    'id': 'test_apple_launch_001',
    'competitor': 'Apple',
    'event_type': 'launch',
    'text': 'Apple announces iPhone 16 with improved camera pipeline, A18 chipset efficiency, and carrier trade-in promos in KSA and UAE; early-bird discounts via select retailers.',
    'impact_score': 8.0,
    'source': 'press_release',
    'timestamp': datetime.now(timezone.utc)
}


analysis = await analyst.analyze_signal(test_signal)

print("=== STRATEGIC ANALYSIS (Mobile OEM) ===")
print("Competitor:", analysis.competitor)
print("Event:", analysis.event_summary, "\n")
print("Strategic Context:\n", analysis.strategic_context, "\n")
if analysis.threat_assessment:
    print("Threat Level:", analysis.threat_assessment.threat_level.value.upper())
    print("Threat Desc:", analysis.threat_assessment.threat_description[:200], "...\n")
if analysis.opportunity_assessment:
    print("Opportunity Level:", analysis.opportunity_assessment.opportunity_level.value.upper())
    print("Opportunity Desc:", analysis.opportunity_assessment.opportunity_description[:200], "...\n")
print("Broader Trends:")
for t in analysis.broader_trends:
    print("-", t)
print("\nRecommendations:")
for r in analysis.strategic_recommendations:
    print("-", r)
print(f"\nCost (approx): ${analysis.llm_cost:.4f} | Tokens: {analysis.tokens_used}")